In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# Basic Filters

In [2]:
def basic_filters(data, trt1m=False,curcd=False):
    data = data[(~data['sic'].between(6000,6999))& #FINANCIALS
                #(data['sic']!=2834)& #PHARMA
                (data['fic']=='USA') &
                (data['exchg'].between(11,19))]
    if trt1m:
        data = data[(~data['trt1m'].isnull()) &
                    (~data['trt1m'].isna()) &
                    (~data['trt1m'].astype(str).str.contains('[a-zA-Z]')) &
                    (data['trt1m']>-100)&
                    (data['trt1m']<1000)]
    if curcd:
        data = data[(data['curcd']=='USD')]
    return data
    

# Fundamentals data

In [3]:
fundamentals = pd.read_csv('fundamentals_annual.csv')
# Get year data
fundamentals['datadate'] = pd.to_datetime(fundamentals['datadate'], format='%Y-%m-%d')
fundamentals['fyear'] = fundamentals['datadate'].dt.year
fundamentals.sort_values(by=['LPERMNO','datadate'],inplace=True)
# Apply basic filters
fundamentals_filtered = basic_filters(fundamentals, trt1m=False,curcd=True)
#Create market cap | fill missings with 0 | filter market caps greater than 10 to avoid outliers
fundamentals_filtered['market_cap'] = fundamentals_filtered['csho']*fundamentals_filtered['prcc_f']
fundamentals_filtered['market_cap'] = fundamentals_filtered['market_cap'].fillna(0)
#fundamentals_filtered = fundamentals_filtered[fundamentals_filtered['market_cap']>10]
#Drop duplicates 
fundamentals_filtered.sort_values(by=['LPERMNO','datadate'],ascending=False, inplace=True) 
fundamentals_filtered.drop_duplicates(subset=['LPERMNO','fyear'],inplace=True)
print("Len of original df:",len(fundamentals),"\nLen of filtered df:",len(fundamentals_filtered),"\nObs removed:",len(fundamentals)-len(fundamentals_filtered),"(",(len(fundamentals)-len(fundamentals_filtered))/len(fundamentals),")")
#Drop 2024
fundamentals_filtered = fundamentals_filtered[fundamentals_filtered['fyear']<2024]
fundamentals_filtered.head()


Len of original df: 51539 
Len of filtered df: 29184 
Obs removed: 22355 ( 0.4337491996352277 )


,GVKEY,LPERMNO,LINKDT,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,...,auopic,ceoso,cfoso,city,conml,gind,sic,state,ipodate,market_cap
47727,184996,93436,2010-06-29,2023-12-31,2023,INDL,C,D,STD,TSLA,...,1.0,Y,Y,Austin,Tesla Inc,251020.0,3711,TX,2010-06-29,791408.80
47726,184996,93436,2010-06-29,2022-12-31,2022,INDL,C,D,STD,TSLA,...,1.0,Y,Y,Austin,Tesla Inc,251020.0,3711,TX,2010-06-29,389741.52
47725,184996,93436,2010-06-29,2021-12-31,2021,INDL,C,D,STD,TSLA,...,1.0,Y,Y,Austin,Tesla Inc,251020.0,3711,TX,2010-06-29,1091653.74
47724,184996,93436,2010-06-29,2020-12-31,2020,INDL,C,D,STD,TSLA,...,1.0,Y,Y,Austin,Tesla Inc,251020.0,3711,TX,2010-06-29,677443.20
47723,184996,93436,2010-06-29,2019-12-31,2019,INDL,C,D,STD,TSLA,...,1.0,Y,Y,Austin,Tesla Inc,251020.0,3711,TX,2010-06-29,75717.73


In [4]:
fundamentals_filtered.groupby(['fyear'])['LPERMNO'].count()

fyear
2014    2784
2015    3095
2016    3015
2017    2989
2018    2987
2019    2966
2020    3061
2021    3428
2022    3380
2023    1476
Name: LPERMNO, dtype: int64

In [5]:
#fundamentals_small = fundamentals_filtered[fundamentals_filtered['revt']<=5.8]
def getSmaller1k(data):
    return data.sort_values(by='revt').head(1000)

fundamentals_small = fundamentals_filtered.groupby('fyear').apply(getSmaller1k)
fundamentals_small = fundamentals_small.reset_index(drop=True)
fundamentals_small.groupby(['fyear'])['LPERMNO'].count()


fyear
2014    1000
2015    1000
2016    1000
2017    1000
2018    1000
2019    1000
2020    1000
2021    1000
2022    1000
2023    1000
Name: LPERMNO, dtype: int64

In [6]:
fundamentals_small['revt'].quantile([0,0.01,0.05,0.1,0.15,0.2,0.25,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99,1])
                                    

0.00   -1964.99900
0.01       0.00000
0.05       0.00000
0.10       0.00000
0.15       0.00000
0.20       0.03100
0.25       0.53750
0.40       8.27840
0.50      18.52900
0.60      31.80000
0.70      50.28840
0.80      86.26760
0.90     143.02430
0.95     822.64160
0.99    3315.46494
1.00    4320.57700
Name: revt, dtype: float64

In [7]:
fundamentals_small = fundamentals_small[(fundamentals_small['revt']>fundamentals_small['revt'].quantile(0.01))&
                                        (fundamentals_small['revt']<=fundamentals_small['revt'].quantile(0.9))]

In [8]:
fundamentals_small.groupby(['fyear'])['LPERMNO'].count()

fyear
2014    776
2015    835
2016    823
2017    804
2018    779
2019    757
2020    733
2021    678
2022    680
2023    183
Name: LPERMNO, dtype: int64

In [11]:
fundamentals_small[['tic','conm','conml','sic']].to_csv('small_companies.csv',index=False)

In [12]:
fundamentals_small.to_csv('small_companies_variables.csv')

# Filter some variables